In [ ]:
%%capture

#import required packages
#basics
import pandas as pd 
import numpy as np
import os
import pickle
import json

#misc
import gc
import time
import warnings

#nlp
import string
import re  
import nltk
nltk.download('popular')  
from nltk.corpus import stopwords
import spacy
from nltk import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
#from KaggleWord2VecUtility import KaggleWord2VecUtility
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer   

lem = WordNetLemmatizer()
ps = PorterStemmer()
tokenizer=TweetTokenizer()
eng_stopwords = set(stopwords.words("english"))

#FeatureEngineering
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_is_fitted
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

#models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Lasso
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.kernel_ridge import KernelRidge
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesRegressor,GradientBoostingRegressor,RandomForestRegressor,VotingClassifier,AdaBoostRegressor
from sklearn.svm import LinearSVR,SVR
from xgboost import XGBRegressor
from sklearn.neural_network import MLPClassifier


#keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils import data

#eval
from sklearn.model_selection import cross_val_score,KFold,GridSearchCV,RandomizedSearchCV,StratifiedKFold,train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Plotly imports
import pandas_profiling as pp
from google.colab import files

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

#settings
start_time=time.time()
warnings.filterwarnings("ignore")

%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
cd drive/My Drive/sapio

/content/drive/My Drive/sapio


In [ ]:
df = pd.read_excel('Lucknow Master.xlsx')
df

,CHECK,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,SOCIAL,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47,Unnamed: 48,Unnamed: 49,Unnamed: 50,Unnamed: 51,Unnamed: 52,Unnamed: 53,Unnamed: 54,Unnamed: 55,Unnamed: 56,Unnamed: 57,Unnamed: 58,Unnamed: 59,Unnamed: 60,Unnamed: 61,Unnamed: 62,% GDP Contribution,Unnamed: 64,Unnamed: 65,Unnamed: 66,Unnamed: 67,Unnamed: 68,Unnamed: 69,Unnamed: 70,Unnamed: 71
0,Check Demographic,Check Social,Ward Number,AREA(IN Sq km),AREA(IN Sq km)_Source,Population( in thousands),Population( in thousands)_Source,Density per square kilometer,Density per square kilometer_Source,Senior citizen %,Senior citizen %_Source,No of cases,No of cases_Source,Educational institutes,Educational institutes_Source,Hospitals,Hospitals_Source,Supermarts/malls,Supermarts/malls_Source,Type,Type_Source,no of households,no of households_Source,Number of Notified Households,Number of Notified Households_Source,No. of household with water connection inside ...,No. of household with water connection inside ...,Total number of water connections,Total number of water connections_Source,No of households having private washroom,No of households having private washroom_Source,Density of retail store,Density of retail store_Source,Per capita income,Per capita income_Source,Labour settlements in the region,Labour settlements in the region_Source,Migrant Population %,Migrant Population %_Source,Gender Ratio,Gender Ratio_Source,Domestic Violence %,Domestic Violence %_Source,Internet Penetration %,Internet Penetration %_Source,Police to Population Ratio per 1000,Police to Population Ratio per 1000_Source,Number of Kirana Shops,Number of Kirana Shops_Source,Number of Pharmacies,Number of Pharmacies_Source,Number of Liquor Stores,Number of Liquor Stores_Source,Estimates of Alcoholism % in Population,Estimates of Alcoholism % in Population_Source,% Unemployed(55%),% Unemployed(55%)_Source,Age Group Children %,Age Group Children %_Source,Literacy Rate,Literacy Rate_Source,Slum percentage,Slum percentage_Source,Agriculture and Food,Manufacturing,Mining,Electricity Gas Water,Construction,Trade Hotels Restaurants,Transport Storage Communication,Financing Real Estate Business Services,Community Social Public Admin
1,#REF!,no of households | Total number of water conne...,1,5.362,approximation,55790,NaN,10404.7,NaN,0.077,Government,5,as on 17/05,84,http://schoolreportcards.in/SRC-New/LocateScho...,9,https://www.healthfrog.in/hospital/list/uttar-...,5,Google map and just dial,NaN,NaN,12398,Approximation,9758,Approximation,6444,Approximation,NaN,NaN,9263,https://smartcities.data.gov.in/resources/publ...,1.3981,NaN,104016,https://www.statista.com/study/66279/lucknow/,NaN,NaN,0.37,https://www.hindustantimes.com/delhi-news/migr...,0.921,NaN,0.0987263,Others,0.76,Others,1.06881,Media,78,https://www.livemint.com/industry/retail/the-n...,34,census 2011,9,http://www.upexciseonline.in/portal/Home/Home/...,0.6238,Other,0.085,Media,0.154,census,0.84,NaN,0.57,NaN,0.136892,0.24321,0.00122911,0.0402533,0.3019,0.092644,0.142269,0.38256,0.195121
2,Type |,no of households | Total number of water conne...,2,5.2,total area 349 sq km,54101,NaN,10404,NaN,0.077,NaN,5,Media,82,NaN,9,NaN,5,NaN,NaN,NaN,12022,NaN,9463,NaN,6249,NaN,NaN,NaN,8982,https://smartcities.data.gov.in/resources/publ...,1.64507,NaN,104016,NaN,NaN,NaN,0.37,NaN,0.891,NaN,0.0987263,Others,0.76,Others,1.06881,NaN,89,https://thewire.in/business/india-retail-secto...,38,NaN,10,NaN,0.6238,NaN,0.085,NaN,0.154,NaN,0.76,NaN,0.53,NaN,0.132748,0.235847,0.0011919,0.03903

In [ ]:
df  = pd.DataFrame(df.values[1:], columns=df.iloc[0])
df.head()

,Check Demographic,Check Social,Ward Number,AREA(IN Sq km),AREA(IN Sq km)_Source,Population( in thousands),Population( in thousands)_Source,Density per square kilometer,Density per square kilometer_Source,Senior citizen %,Senior citizen %_Source,No of cases,No of cases_Source,Educational institutes,Educational institutes_Source,Hospitals,Hospitals_Source,Supermarts/malls,Supermarts/malls_Source,Type,Type_Source,no of households,no of households_Source,Number of Notified Households,Number of Notified Households_Source,No. of household with water connection inside the house,No. of household with water connection inside the house_Source,Total number of water connections,Total number of water connections_Source,No of households having private washroom,No of households having private washroom_Source,Density of retail store,Density of retail store_Source,Per capita income,Per capita income_Source,Labour settlements in the region,Labour settlements in the region_Source,Migrant Population %,Migrant Population %_Source,Gender Ratio,Gender Ratio_Source,Domestic Violence %,Domestic Violence %_Source,Internet Penetration %,Internet Penetration %_Source,Police to Population Ratio per 1000,Police to Population Ratio per 1000_Source,Number of Kirana Shops,Number of Kirana Shops_Source,Number of Pharmacies,Number of Pharmacies_Source,Number of Liquor Stores,Number of Liquor Stores_Source,Estimates of Alcoholism % in Population,Estimates of Alcoholism % in Population_Source,% Unemployed(55%),% Unemployed(55%)_Source,Age Group Children %,Age Group Children %_Source,Literacy Rate,Literacy Rate_Source,Slum percentage,Slum percentage_Source,Agriculture and Food,Manufacturing,Mining,Electricity Gas Water,Construction,Trade Hotels Restaurants,Transport Storage Communication,Financing Real Estate Business Services,Community Social Public Admin
0,#REF!,no of households | Total number of water conne...,1,5.362,approximation,55790,NaN,10404.7,NaN,0.077,Government,5,as on 17/05,84,http://schoolreportcards.in/SRC-New/LocateScho...,9,https://www.healthfrog.in/hospital/list/uttar-...,5,Google map and just dial,NaN,NaN,12398,Approximation,9758,Approximation,6444,Approximation,NaN,NaN,9263,https://smartcities.data.gov.in/resources/publ...,1.3981,NaN,104016,https://www.statista.com/study/66279/lucknow/,NaN,NaN,0.37,https://www.hindustantimes.com/delhi-news/migr...,0.921,NaN,0.0987263,Others,0.76,Others,1.06881,Media,78,https://www.livemint.com/industry/retail/the-n...,34,census 2011,9,http://www.upexciseonline.in/portal/Home/Home/...,0.6238,Other,0.085,Media,0.154,census,0.84,NaN,0.57,NaN,0.136892,0.24321,0.00122911,0.0402533,0.3019,0.092644,0.142269,0.38256,0.195121
1,Type |,no of households | Total number of water conne...,2,5.2,total area 349 sq km,54101,NaN,10404,NaN,0.077,NaN,5,Media,82,NaN,9,NaN,5,NaN,NaN,NaN,12022,NaN,9463,NaN,6249,NaN,NaN,NaN,8982,https://smartcities.data.gov.in/resources/publ...,1.64507,NaN,104016,NaN,NaN,NaN,0.37,NaN,0.891,NaN,0.0987263,Others,0.76,Others,1.06881,NaN,89,https://thewire.in/business/india-retail-secto...,38,NaN,10,NaN,0.6238,NaN,0.085,NaN,0.154,NaN,0.76,NaN,0.53,NaN,0.132748,0.235847,0.0011919,0.0390347,0.29276,0.0898393,0.137962,0.370978,0.189214
2,Type |,no of households | Total number of water conne...,3,2.229,NaN,23187,NaN,10402.4,NaN,0.077,NaN,2,NaN,35,NaN,4,NaN,2,NaN,NaN,NaN,5153,NaN,4056,NaN,2679,NaN,NaN,NaN,3850,https://smartcities.data.gov.in/resources/publ...,2.97581,NaN,104016,NaN,NaN,NaN,0.37,NaN,0.962,NaN,0.0987263,Others,0.76,Others,1.06881,NaN,69,NaN,30,NaN,8,NaN,0.6238,NaN,0.085,NaN,0.154,NaN,0.74,NaN,0.893,NaN,0.0568939,0.101081,0.000510832,0.0167298,0.125473,0.038504,0.0591288,0.158997,0.0810946
3,Type |,no of households | Total number of water conne...,4,2.814,NaN,29282,NaN,10405.8,NaN,0.077,NaN,2,NaN,44,NaN,5,NaN,3,NaN,NaN,NaN,6507,NaN,5122,NaN,3382,NaN,NaN,NaN,4862,https://smartcities.data.gov.in/resources/publ...,5.56656,NaN,104016,NaN,NaN,NaN,0.37,NaN,0.892,NaN,0.0987263,Others,0.76,Others,1.0688

In [ ]:

df = df.drop(columns=['Check Demographic','Check Social'])
df.head()

for col in df.columns: 
    if 'Source' in col: 
        df.drop(columns=col, inplace=True)
df.head() 

,Ward Number,AREA(IN Sq km),Population( in thousands),Density per square kilometer,Senior citizen %,No of cases,Educational institutes,Hospitals,Supermarts/malls,Type,no of households,Number of Notified Households,No. of household with water connection inside the house,Total number of water connections,No of households having private washroom,Density of retail store,Per capita income,Labour settlements in the region,Migrant Population %,Gender Ratio,Domestic Violence %,Internet Penetration %,Police to Population Ratio per 1000,Number of Kirana Shops,Number of Pharmacies,Number of Liquor Stores,Estimates of Alcoholism % in Population,% Unemployed(55%),Age Group Children %,Literacy Rate,Slum percentage,Agriculture and Food,Manufacturing,Mining,Electricity Gas Water,Construction,Trade Hotels Restaurants,Transport Storage Communication,Financing Real Estate Business Services,Community Social Public Admin
0,1,5.362,55790,10404.7,0.077,5,84,9,5,NaN,12398,9758,6444,NaN,9263,1.3981,104016,NaN,0.37,0.921,0.0987263,0.76,1.06881,78,34,9,0.6238,0.085,0.154,0.84,0.57,0.136892,0.24321,0.00122911,0.0402533,0.3019,0.092644,0.142269,0.38256,0.195121
1,2,5.2,54101,10404,0.077,5,82,9,5,NaN,12022,9463,6249,NaN,8982,1.64507,104016,NaN,0.37,0.891,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.76,0.53,0.132748,0.235847,0.0011919,0.0390347,0.29276,0.0898393,0.137962,0.370978,0.189214
2,3,2.229,23187,10402.4,0.077,2,35,4,2,NaN,5153,4056,2679,NaN,3850,2.97581,104016,NaN,0.37,0.962,0.0987263,0.76,1.06881,69,30,8,0.6238,0.085,0.154,0.74,0.893,0.0568939,0.101081,0.000510832,0.0167298,0.125473,0.038504,0.0591288,0.158997,0.0810946
3,4,2.814,29282,10405.8,0.077,2,44,5,3,NaN,6507,5122,3382,NaN,4862,5.56656,104016,NaN,0.37,0.892,0.0987263,0.76,1.06881,163,70,18,0.6238,0.085,0.154,0.77,1.582,0.0718492,0.127651,0.000645111,0.0211274,0.158455,0.0486252,0.0746716,0.200791,0.102411
4,5,2.72,28306,10406.6,0.077,2,43,4,3,NaN,6290,4951,3270,NaN,4832,3.14421,104016,NaN,0.37,0.899,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.69,0.804,0.0694544,0.123397,0.000623609,0.0204232,0.153174,0.0470045,0.0721827,0.194098,0.0989979


In [ ]:
def Encoder(val):
    if val in category:
        return category[val]
    else:
        category[val]=len(category)
    return category[val]

l1 = ['Type','Labour settlements in the region']

i=0
for col in df.columns:
    category={}
    if (col in l1):
        df.iloc[:,i]=df.iloc[:,i].apply(Encoder)
        print(col)
    i+=1

Type
Labour settlements in the region


In [ ]:

df

,Ward Number,AREA(IN Sq km),Population( in thousands),Density per square kilometer,Senior citizen %,No of cases,Educational institutes,Hospitals,Supermarts/malls,Type,no of households,Number of Notified Households,No. of household with water connection inside the house,Total number of water connections,No of households having private washroom,Density of retail store,Per capita income,Labour settlements in the region,Migrant Population %,Gender Ratio,Domestic Violence %,Internet Penetration %,Police to Population Ratio per 1000,Number of Kirana Shops,Number of Pharmacies,Number of Liquor Stores,Estimates of Alcoholism % in Population,% Unemployed(55%),Age Group Children %,Literacy Rate,Slum percentage,Agriculture and Food,Manufacturing,Mining,Electricity Gas Water,Construction,Trade Hotels Restaurants,Transport Storage Communication,Financing Real Estate Business Services,Community Social Public Admin
0,1,5.362,55790,10404.7,0.077,5,84,9,5,0,12398,9758,6444,NaN,9263,1.3981,104016,0,0.37,0.921,0.0987263,0.76,1.06881,78,34,9,0.6238,0.085,0.154,0.84,0.57,0.136892,0.24321,0.00122911,0.0402533,0.3019,0.092644,0.142269,0.38256,0.195121
1,2,5.2,54101,10404,0.077,5,82,9,5,0,12022,9463,6249,NaN,8982,1.64507,104016,0,0.37,0.891,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.76,0.53,0.132748,0.235847,0.0011919,0.0390347,0.29276,0.0898393,0.137962,0.370978,0.189214
2,3,2.229,23187,10402.4,0.077,2,35,4,2,0,5153,4056,2679,NaN,3850,2.97581,104016,0,0.37,0.962,0.0987263,0.76,1.06881,69,30,8,0.6238,0.085,0.154,0.74,0.893,0.0568939,0.101081,0.000510832,0.0167298,0.125473,0.038504,0.0591288,0.158997,0.0810946
3,4,2.814,29282,10405.8,0.077,2,44,5,3,0,6507,5122,3382,NaN,4862,5.56656,104016,0,0.37,0.892,0.0987263,0.76,1.06881,163,70,18,0.6238,0.085,0.154,0.77,1.582,0.0718492,0.127651,0.000645111,0.0211274,0.158455,0.0486252,0.0746716,0.200791,0.102411
4,5,2.72,28306,10406.6,0.077,2,43,4,3,0,6290,4951,3270,NaN,4832,3.14421,104016,0,0.37,0.899,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.69,0.804,0.0694544,0.123397,0.000623609,0.0204232,0.153174,0.0470045,0.0721827,0.194098,0.0989979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,106,2.364,24597,10404.8,0.077,2,37,4,2,0,5466,4302,2841,NaN,1787,2.39867,104016,0,0.37,0.941,0.0987263,0.76,1.06881,59,25,7,0.6238,0.085,0.154,0.75,0,0.0603537,0.107228,0.000541896,0.0177471,0.133103,0.0408454,0.0627244,0.168665,0.086026
106,107,2.939,30575,10403.2,0.077,3,46,5,3,0,6794,5348,3532,NaN,6028,3.82666,104016,0,0.37,0.938,0.0987263,0.76,1.06881,117,50,13,0.6238,0.085,0.154,0.74,0.0736,0.0750219,0.133288,0.000673597,0.0220603,0.165452,0.0507724,0.0779689,0.209657,0.106934
107,108,1.511,15719,10403,0.077,1,24,2,1,0,3493,2749,1816,NaN,3099,2.99001,104016,0,0.37,0.985,0.0987263,0.76,1.06881,47,20,5,0.6238,0.085,0.154,0.77,1.094,0.0385697,0.0685251,0.000346305,0.0113415,0.0850611,0.0261027,0.0400848,0.107787,0.0549759
108,109,2.872,29878,10403.2,0.077,3,45,5,3,0,6640,5226,3451,NaN,5891,1.53959,104016,0,0.37,0.94,0.0987263,0.76,1.06881,46,20,5,0.6238,0.085,0.154,0.76,0,0.0733116,0.13025,0.000658241,0.0215574,0.161681,0.049615,0.0761915,0.204878,0.104496


In [ ]:


df.drop(['Labour settlements in the region'],axis=1,inplace=True)


In [ ]:

df.drop(['Ward Number'],axis=1,inplace=True)

In [ ]:

df

,AREA(IN Sq km),Population( in thousands),Density per square kilometer,Senior citizen %,No of cases,Educational institutes,Hospitals,Supermarts/malls,Type,no of households,Number of Notified Households,No. of household with water connection inside the house,Total number of water connections,No of households having private washroom,Density of retail store,Per capita income,Migrant Population %,Gender Ratio,Domestic Violence %,Internet Penetration %,Police to Population Ratio per 1000,Number of Kirana Shops,Number of Pharmacies,Number of Liquor Stores,Estimates of Alcoholism % in Population,% Unemployed(55%),Age Group Children %,Literacy Rate,Slum percentage,Agriculture and Food,Manufacturing,Mining,Electricity Gas Water,Construction,Trade Hotels Restaurants,Transport Storage Communication,Financing Real Estate Business Services,Community Social Public Admin
0,5.362,55790,10404.7,0.077,5,84,9,5,0,12398,9758,6444,NaN,9263,1.3981,104016,0.37,0.921,0.0987263,0.76,1.06881,78,34,9,0.6238,0.085,0.154,0.84,0.57,0.136892,0.24321,0.00122911,0.0402533,0.3019,0.092644,0.142269,0.38256,0.195121
1,5.2,54101,10404,0.077,5,82,9,5,0,12022,9463,6249,NaN,8982,1.64507,104016,0.37,0.891,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.76,0.53,0.132748,0.235847,0.0011919,0.0390347,0.29276,0.0898393,0.137962,0.370978,0.189214
2,2.229,23187,10402.4,0.077,2,35,4,2,0,5153,4056,2679,NaN,3850,2.97581,104016,0.37,0.962,0.0987263,0.76,1.06881,69,30,8,0.6238,0.085,0.154,0.74,0.893,0.0568939,0.101081,0.000510832,0.0167298,0.125473,0.038504,0.0591288,0.158997,0.0810946
3,2.814,29282,10405.8,0.077,2,44,5,3,0,6507,5122,3382,NaN,4862,5.56656,104016,0.37,0.892,0.0987263,0.76,1.06881,163,70,18,0.6238,0.085,0.154,0.77,1.582,0.0718492,0.127651,0.000645111,0.0211274,0.158455,0.0486252,0.0746716,0.200791,0.102411
4,2.72,28306,10406.6,0.077,2,43,4,3,0,6290,4951,3270,NaN,4832,3.14421,104016,0.37,0.899,0.0987263,0.76,1.06881,89,38,10,0.6238,0.085,0.154,0.69,0.804,0.0694544,0.123397,0.000623609,0.0204232,0.153174,0.0470045,0.0721827,0.194098,0.0989979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,2.364,24597,10404.8,0.077,2,37,4,2,0,5466,4302,2841,NaN,1787,2.39867,104016,0.37,0.941,0.0987263,0.76,1.06881,59,25,7,0.6238,0.085,0.154,0.75,0,0.0603537,0.107228,0.000541896,0.0177471,0.133103,0.0408454,0.0627244,0.168665,0.086026
106,2.939,30575,10403.2,0.077,3,46,5,3,0,6794,5348,3532,NaN,6028,3.82666,104016,0.37,0.938,0.0987263,0.76,1.06881,117,50,13,0.6238,0.085,0.154,0.74,0.0736,0.0750219,0.133288,0.000673597,0.0220603,0.165452,0.0507724,0.0779689,0.209657,0.106934
107,1.511,15719,10403,0.077,1,24,2,1,0,3493,2749,1816,NaN,3099,2.99001,104016,0.37,0.985,0.0987263,0.76,1.06881,47,20,5,0.6238,0.085,0.154,0.77,1.094,0.0385697,0.0685251,0.000346305,0.0113415,0.0850611,0.0261027,0.0400848,0.107787,0.0549759
108,2.872,29878,10403.2,0.077,3,45,5,3,0,6640,5226,3451,NaN,5891,1.53959,104016,0.37,0.94,0.0987263,0.76,1.06881,46,20,5,0.6238,0.085,0.154,0.76,0,0.0733116,0.13025,0.000658241,0.0215574,0.161681,0.049615,0.0761915,0.204878,0.104496


In [ ]:
df.drop(['Migrant Population %'],axis=1,inplace=True)

In [ ]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col])
    print(col)

AREA(IN Sq km)
Population( in thousands)
Density per square kilometer
Senior citizen %
No of cases
Educational institutes
Hospitals
Supermarts/malls
Type
no of households
Number of Notified Households
No. of household with water connection inside the house
Total number of water connections
No of households having private washroom
Density of retail store
Per capita income
Gender Ratio
Domestic Violence %
Internet Penetration %
Police to Population Ratio per 1000
Number of Kirana Shops
Number of Pharmacies
Number of Liquor Stores
Estimates of Alcoholism % in Population
% Unemployed(55%)
Age Group Children %
Literacy Rate
Slum percentage
Agriculture and Food
Manufacturing
Mining
Electricity Gas Water
Construction
Trade Hotels Restaurants
Transport Storage Communication
Financing Real Estate Business Services
Community Social Public Admin


In [ ]:
if pd.__version__!='0.25.3':
  !pip uninstall pandas
  !pip install pandas==0.25.3

Uninstalling pandas-1.0.4:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/pandas-1.0.4.dist-info/*
    /usr/local/lib/python3.6/dist-packages/pandas/*
Proceed (y/n)? y
  Successfully uninstalled pandas-1.0.4
     |████████████████████████████████| 10.4MB 2.6MB/s 
ERROR: google-colab 1.0.0 has requirement pandas~=1.0.0; python_version >= "3.0", but you'll have pandas 0.25.3 which is incompatible.


In [ ]:
print( pd.__version__)

0.25.3


In [ ]:
try:
  if pandas_profiling.__version__!='2.8.0':
    !pip uninstall pandas_profiling
    !pip install 'pandas_profiling==2.8.0'
except NameError:
  !pip install 'pandas_profiling==2.8.0'


     |████████████████████████████████| 266kB 2.8MB/s 
     |████████████████████████████████| 61kB 6.3MB/s 
     |████████████████████████████████| 71kB 6.4MB/s 
     |████████████████████████████████| 604kB 6.9MB/s 
     |████████████████████████████████| 3.1MB 15.1MB/s 
     |████████████████████████████████| 296kB 32.9MB/s 
  Created wheel for confuse: filename=confuse-1.1.0-cp36-none-any.whl size=17574 sha256=704e6628352cc8280c6bd5760ea51d3fa9cc6045bee19021935d22fffd710126
  Stored in directory: /root/.cache/pip/wheels/f6/8b/23/41a1b516f6d8d4cc81f5bdb55394a47cdbe9659c53668d3c9e
  Created wheel for htmlmin: filename=htmlmin-0.1.12-cp36-none-any.whl size=27084 sha256=668edc9bfd3b4d8e9781d1c7577043678ab0a232183435306c63c658632a127e
  Stored in directory: /root/.cache/pip/wheels/43/07/ac/7c5a9d708d65247ac1f94066cf1db075540b85716c30255459
  Created wheel for imagehash: filename=ImageHash-4.1.0-py2.py3-none-any.whl size=291990 sha256=fd142295a4f12ef089906b7ec7743e22fd19456a1e30a87e111d5

In [ ]:
profile=pp.ProfileReport(df,title="Lucknow master",explorative=True)

In [ ]:
profile.to_file("lucknow Master"+".html")
files.download("lucknow Master"+".html")

In [ ]:
df.to_csv('Lucknow Master.csv',index=False)
files.download("Lucknow Master.csv") 

In [ ]:
pred_col=['Educational institutes','Hospitals','Number of Households','Number of Notified Households','No. of household with water connection inside the house','No of households having private washroom','Density of retail store','Domestic Violence %','Police to Population Ratio per 1000','Number of Kirana Shops','Domestic Violence %','Number of Pharmacies','% Unemployed(55%)','Slum percentage']

In [ ]:
for col in pred_col:
  X=df.loc[:,df.columns!=col]
  y=df.loc[:,col]
  from sklearn.model_selection import train_test_split
  train_X,test_X,train_y,test_y=train_test_split(X,y,random_state=0,test_size=0.25)
  import xgboost 
  reg=xgboost.XGBRegressor(n_estimator=1000)
  reg.fit(train_X,train_y)
  pred_y=reg.predict(test_X)
  check=pd.DataFrame({"Actual "+col:test_y,"Predicted "+col:np.round(pred_y)})
  print(check)
  from sklearn.metrics import r2_score
  from sklearn.metrics import max_error
  maxerr=max_error(test_y,pred_y)
  print(f"Maximum Error={maxerr}")
  R2=r2_score(test_y,pred_y)
  print(f"R^2={R2}")
  n=X.shape[0]
  p=X.shape[1]
  Adj_R2=1-(1-R2)*(n-1)/(n-p-1)
  print(f"Adjusted R^2={Adj_R2}")
  print("------------------------------------------------------------------------------------------------------------")

[16:21:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
     Actual Educational institutes  Predicted Educational institutes
85                              48                              48.0
10                             106                             108.0
75                              39                              39.0
2                               35                              35.0
24                              69                              69.0
101                             28                              28.0
109                             36                              36.0
7                               50                              49.0
16                              31                              32.0
91                              54                              53.0
68                              48                              49.0
22                              38           

KeyError: ignored